Метод опорных векторов (support vector machine, SVM) был разработан во второй половине XX века ([1963]($On a perceptron class$), [1992]($A Training Algorithm for Optimal Margin Classifiers$), [1995]($Support-Vector Networks$)) и получил широкое распространение в машинном обучении. Он относится к классу так называемых ядерных методов (kernel methods), в которых используются функции особого вида, называемые ядрами.

В этом обзоре мы кратко рассмотрим метод опорных векторов и понятие ядер.

Пусть мы решаем задачу бинарной классификации $\mathbb{R}^n \to \{0, 1\}$. Метод опорных векторов основан на понятии *разделяющей гиперплоскости* в пространстве признаков: $w \cdot x + b = 0$ ($w$ - вектор, $b$ - скаляр) Эта плоскость выбирается так, что объекты одного класса лежат с одной стороны от нее ($w \cdot x + b > 0$), а объекты другого класса лежат с другой стороны ($w \cdot x + b < 0$). Конечно такая плоскость не всегда существует, поэтому рассмотрим последовательно три случая.

### Случай 1. Линейно разделимая выборка

Если для обучающей выборки существует разделяющая гиперплоскость, то она как правило не единствена.

Идея метода опорных векторов в том, что мы ищем пару параллельных плоскостей, на одной из которых лежат некоторые элементы класса $1$, а на другой лежат некоторые элементы класса $0$, при этом между плоскостями нет элементов ни того, ни другого класса. При этом мы выбираем плоскости так, чтобы расстояние между ними ("зазор между классами") было максимально. Если выборка линейно разделима, то такое решение всегда единствено.

Без ограничения общности можно считать, что плоскость, на которой лежат объекты класса $y = 1$ задается уравнением $w \cdot x - b = 1$, а плоскость, на которой лежат объекты класса $y = 0$, задается уравнением $w \cdot x - b = -1$.

<img src="assets/1024px-SVM_margin.png" width="400" align="center">

<center><i>Рис. из <a href=https://en.wikipedia.org/wiki/Support-vector_machine>Википедии</a></i></center>

Задача сводится к тому, чтобы найти наименьший по модулю вектор $w$ при заданных ограничениях, который соответствует наибольшему зазору между классами. Для выборки размера $N$ получается следующая задача оптимизации с неизвестными $w$ и $b$:

$\begin{cases}
    \|w\| \to \underset{w, b}{\text{min}}\\
    y_i (w \cdot x_i - b) \geq 1 & i = 1, \dots, N
\end{cases}$

Такая задача оптимизации является [выпуклой](https://www.quora.com/Why-is-SVM-convex-optimization) и имеет единственное решение. Разделяющей гиперплоскостью при этом считаем мы плоскость $w \cdot x - b = 0$ (обозначена красным на рисунке). Классификация произвольного $x$ (инференс SVM) осуществляется следующим образом:

$y = \text{sign}(w \cdot x - b) \tag{1}$

### Двойственная задача

Для сформулированной выше задачи оптимизации можно построить двойственную задачу и тем самым показать, что решение $w$ будет линейно выражаться через опорные вектора $\{x_i\}$:

$w = \sum\limits_i \alpha_i x_i$

$\sum\limits_i \alpha_i = 0$

Поскольку скалярное произведение линейно, то классификацию произвольного $x$ можно осуществлять, считая скалярные произведения $x$ с опорными векторами:

$y = \text{sign}(w \cdot x - b) = \text{sign}\Big(\big(\sum\limits_i \alpha_i x_i\big) \cdot x - b\Big) = \text{sign}\Big( \sum\limits_i \alpha_i (x_i \cdot x ) - b\Big)$

Формулировка двойственной задачи оптимизации здесь приводиться не будет. Важно лишь то, что решив двойственную задачу и найдя опорные вектора $\{x_i\}$ и их коэффициенты $\{\alpha_i\}$, мы можем осуществлять классификацию *без непосредственного подсчета $w$*. Этот факт будет использоваться в дальнейшем при переходе к нелинейному SVM.

### SVM и логистическая регрессия

В методе опорных векторов лишь небольшая часть обучающей выборки влияет на решение, а именно только те объекты обучающей выборки, которые лежат на одной из плоскостей. Векторы признаков для этих объектов называются *опорными векторами*. Если удалить из выборки все объекты, кроме опорных, то решение не изменится.

В логистической регрессии, как и в SVM, строится разделяющая гиперплоскость. Однако в логистической регрессии она строится другим способом, при котором каждый элемент выборки оказывает некое влияние на решение. Минимизация нормы $w$ в SVM схожа с L2-регуляризацией, применяемой в методе гребневой регрессии.

Если для обучающей выборки не существует оптимальной разделяющей гиперплоскости, то этому может быть две причины: либо в выборке присутствует случайный шум, либо разделяющая поверхность между классами не является плоскостью. Далее рассмотрим эти два случая.

### Случай 2. Поиск гиперплоскости при линейной неразделимости выборки (soft-margin SVM)

В этом случае разделяющая гиперплоскость не существует из-за шума в выборке, но мы все же предполагаем, что признаки линейно влияют на ответ, и хотим выполнить линейную классификацию. Как бы мы ни построили разделяющую гиперплоскость, всегда будут присутствовать "объекты-нарушители", для которых $y_i (w \cdot x_i - b) < 1$.

Идея заключается в том, что мы вводим для каждой пары из обучающей выборки $(x_i, y_i)$ переменную $\xi_i$, заранее неизвестную. Для объектов-нарушителей $\xi_i = 1 - y_i (w \cdot x_i - b)$, то есть $\xi_i$ равно "величине нарушения", а для остальных объектов $\xi_i = 0$. В процессе оптимизации мы хотим минимизировать как $\|w\|$, так и сумму всех $\xi_i$, в идеале сделав ее нулевой. Поскольку теперь у нас одновременно две цели оптимизации, мы должны ввести коэффициент балансировки $С$ между ними. Мы получаем задачу оптимизации, называемую *soft-margin SVM*:

$\begin{cases}
    \|w\| + C \sum\limits_i \xi_i \to \underset{w, b, \xi}{\text{min}}\\
    y_i (w \cdot x_i - b) \geq 1 - \xi_i & i = 1, \dots, N\\
    \xi_i \geq 0 & i = 1, \dots, N
\end{cases} \tag{2}$

Константа $C$, выбираемая заранее, является компромиссом между шириной зазора и минимизацией суммарной ошибки "объектов-нарушителей". Степень регуляризации [увеличивается](https://datascience.stackexchange.com/questions/25977/using-svm-classifier-with-c-0-and-c-infinity-what-would-be-the-effect-on-classi) при $C \to 0$.

### Случай 3. Нелинейная разделяющая поверхность между классами

Можно применить SVM для поиска нелинейной разделяющей поверхности, если сначала отобразить векторы признаков с помощью некой функции в пространство, в котором классы могут быть разделены гиперплоскостью. Это пространство называется *спрямляющим*. Например, пусть $x \in \mathbb{R}^3$, то есть $x$ - вектор из $3$ чисел (признаков): $x = (v_1, v_2, v_3)$. Обозначим за $P_n(x)$ полином $n$-й степени над множеством признаков (см. [polynomial features](https://scikit-learn.org/stable/modules/preprocessing.html#polynomial-features) в sklearn):

$P_1(x)$ = $(1,\ v_1, v_2, v_3)$

$P_2(x)$ = $(1,\ v_1, v_2, v_3, v_1^2, v_2^2, v_3^2, 2 v_1 v_2, 2 v_1 v_3, 2 v_2 v_3)$

$P_3(x) = \dots$

В общем случае функция отображения в спрямляющее пространство обозначается как $\Phi: \mathcal{L} \to \mathcal{H}$. Исходное и спрямляющее пространство обозначаются $\mathcal{L}$ и $\mathcal{H}$, от слов *low-dimensional* и *high-dimensional*, потому что как правило $\mathcal{H}$ имеет намного большую размерность, чем $\mathcal{L}$, что мы видим на примере полиномов. Конечно, мы не всегда уверены в том, что выборка окажется линейно разделима в пространстве $\mathcal{H}$. Скорее, мы это предполагаем и пробуем. Как мы увидим далее, для некоторых $\Phi$ любая выборка всегда линейно разделима.

### Использование ядер в SVM

В этом обзоре мы не рассматривали подробно двойственную задачи оптимизации в SVM, но в ней важно следующее:

1. Решение двойственной задачи эквивалентно решению исходной задачи оптимизации на переменные $w$ и $b$.
2. В двойственной задаче векторы признаков $x_i$ не используются никаким другим образом, кроме подсчета их попарных скалярных произведений: $x_i \cdot x_j$.

В нелинейном случае мы отображаем векторы признаков функцией $\Phi$ в спрямляющее пространство $\mathcal{H}$, а затем решаем двойственную задачу. В двойственной задаче считаются только попарные скалярные произведения, то есть выражения вида $\Phi(x) \cdot \Phi(x')$. Благодаря такому свойству, мы могли бы заранее найти такую функцию $K$, что:

$\forall x, x' \in \mathcal{L}: K(x, x') = \Phi(x) \cdot \Phi(x') \tag{3}$

Таким образом, мы объединяем $\Phi$ и скалярное произведение в единую операцию. Это избавляет нас от необходимости в явном виде рассчитывать $\Phi(x)$, которое может иметь очень большую размерность. Например, если мы возьмем в качестве $\Phi(x)$ полином $n$-й степени $P_n(x)$, то $K(x, x') = (x \cdot x' + 1)^n$. Это выражение рассчитывается элементарно, тогда как прямой подсчет $\Phi(x)$ и $\Phi(x')$ потребовал бы очень много времени из-за большой размерности $\mathcal{H}$.

Описанный выше прием называется *kernel trick*, а функции $K$ называются *ядрами (kernels)*. Функция  от двух аргументов $K$ является ядром, если существует $\Phi: \mathcal{L} \to \mathcal{H}$ такое, что верно условие $(3)$. Интересно то, что для некоторых ядер $\mathcal{H}$ является бесконечномерным пространством. Например, рассмотрим гауссово радиально-базисное ядро:

$K(x, x') = \exp \Bigg( \cfrac{\|x - x'\|^2}{2\sigma} \Bigg)$

Для этого ядра пространство $\mathcal{H}$ бесконечномерно и прямой подсчет $\Phi(x)$ невозможен. Тем не менее, это не мешает применять ядро RBF в методе опорных векторов. При использовании гауссова радиально-базисного ядра любая выборка всегда линейно разделима, однако линейное разделение может быть сопряжено с излишней сложностью разделяющей поверхности и переобучением. В этом случае для регуляризации можно использовать soft-margin SVM, который не требует 100%-й линейной разделимости.

Kernel trick позволяет решать задачи машинного обучения даже тогда, когда признаков как таковых нет, но известно, как рассчитывать функцию $K(x, x')$ (featureless recognition). В более общем случае, $K$ может не быть ядром (то есть соответствующее $\Phi$ не существует), такие функции называются *non-mercer kernels*. Для них двойственная задача будет невыпуклой, что может привести к трудностям при оптимизации и множеству неоптимальных локальных минимумов.

### Связь SVM с нейронными сетями

Принцип работы SVM заключается в отображении пространства признаков $\mathcal{L}$ в спрямляющее пространство $\mathcal{H}$ и нахождении разделяющей плоскости в этом пространстве. Нейронные сети работают похожим образом: все слои, кроме последнего, можно представить как функцию $\Phi(x)$, которая отображают входные данные в пространство $\mathcal{H}$, где возможно линейное разделение классов. Последний слой сети осуществляет это разделение.

Отличия между нейронными сетями в том, что в нейронных сетях функция $\Phi(x)$ является обучаемой, а пространство $\mathcal{H}$ имеет ограниченную размерность (как правило до 2000). В SVM, напротив, функция $\Phi(x)$ фиксирована, а пространство $\mathcal{H}$ может иметь очень большую или даже бесконечную размерность. Последний слой обучается по-разному в нейронных сетях и SVM. Наконец, в SVM задача оптимизации выпукла и имеет единственное решение.

Алгоритм инференса SVM можно представить как граф вычислений, в котором первый слой рассчитывает скалярные произведения входного вектора с опорными векторами $(1)$, а второй слой осуществляет линейную классификацию. При этом число элементов первого слоя определяется автоматически и равно числу опорных векторов.

<img src="assets/svm.jpg" width="500" align="center">

<center><i>Рис. из <a href=$Математические методы обучения по прецедентам$>лекций К. Воронцова</a> по машинному обучению</i></center>

Если при этом использовать сигмоидное ядро $K(x, x') = \tanh (k_0 + k_1(x \cdot x'))$, то элементы первого слоя можно представить как нейроны в нейронной сети с одним скрытым слоем и функцией активации $\tanh$. Однако в SVM отображение $K(x, x')$ необучаемо, поэтому представление SVM как нейронной сети имеет место только при инференсе, но не при обучении.

### SVM в задаче регрессии

Задача регрессии близка к задаче бинарной классификации. В случае линейной бинарной классификации мы ищем ответ в виде $\text{sign}(w \cdot x - b)$, а в случае линеной регрессии ищем ответ в виде $w \cdot x - b$.

При классификации с помощью SVM ошибкой на $i$-м примере считалось неравенство $y_i (w \cdot x_i - b) < 1$, и переменные $\xi_i$ описывали величину ошибок. При регрессии с помощью SVM ошибкой считается отклонение более чем на $\epsilon$ от предсказанного ответа: $y_i < w \cdot x_i - b - \epsilon$ или $y_i > w \cdot x_i - b + \epsilon$.

Как и в случае классификации, SVM для регрессии можно описывать в двах вариантах: hard-margin и soft-margin. В случае harg-margin требование "классы должны быть разделимы гиперплоскостью" трансформируется для регрессии в требование "предсказания не должны отличаться от верных ответов больше, чем на $\epsilon$". При этом те объекты, на которых предсказания отличаются ровно на $\epsilon$, называются опорными векторами.

В случае soft-margin SVM для классификации мы разрешали некоторым примерам нарушать условие $y_i (w \cdot x_i - b) \geq 1$, но вводили для них соответствующие штрафы $\xi_i$, сумму которых стремились уменьшить. В регрессии при soft-margin SVM мы, по аналогии, разрешаем некоторым примерам отличаться от предсказаний больше, чем на $\epsilon$, и вводим штрафы $\xi_i^+$ и $\xi_i^-$ для отклонений, соответственно, в большую или в меньшую сторону. В итоге мы приходим к следующей задаче оптимизации относительно $w, b, \xi^+, \xi^-$ для выборки $\{x_i, y_i\}_{i=1}^N$ c гиперпараметрами $\epsilon, C$:

$\begin{cases}
    \|w\| + C \sum\limits_i (\xi_i^+ + \xi_i^-) \to \underset{w, b, \xi^+, \xi^-}{\text{min}}\\
    y_i - \epsilon - \xi_i^- \leq w \cdot x_i - b & i = 1, \dots, N\\
    y_i + \epsilon + \xi_i^+ \geq w \cdot x_i - b & i = 1, \dots, N\\
    \xi_i^- \geq 0 & i = 1, \dots, N\\
    \xi_i^+ \geq 0 & i = 1, \dots, N
\end{cases}$

### Связь SVM и метода наименьших квадратов в задаче регрессии

В методе наименьших квадратов функция потерь на $i$-м обучающем примере равна квадрату разности эталонного ответа и предсказания: $\Delta_i^2$, где $\Delta_i = y_i - (w \cdot x_i + b)$. Мы получим hard-margin SVM, если в методе наименьших квадратов добавим к функции потерь слагаемое $\|w\|/C$, а саму функцию потерь $\Delta_i^2$ заменим на следующую:

$\begin{cases}
    0 & -\epsilon \leq \Delta_i \leq \epsilon\\
    \infty & \text{otherwise}
\end{cases}$

Чтобы такая функция потерь не была равна бесконечности, предсказания на всех примерах должны отличаться от эталонных ответов не более, чем на $\epsilon$. Помимо этого требования, мы минимизируем $\|w\|$, что эквивалентно гребневой регрессии.

Мы получим soft-margin SVM, если в методе наименьших квадратов добавим к функции потерь слагаемое $\|w\|/C$, а саму функцию потерь $\Delta_i^2$ заменим на следующую:

$\max(0, |\Delta_i| - \epsilon)$

Функции потерь можно сравнить визуально для метода наименьших квадратов и soft-margin SVM. На графике ниже по горизональной оси отложена ошибка предсказания, по вертикальной значение функции потерь, при $\epsilon = 1$.

<img src="assets/svm2.jpg" width="300" align="center">

<center><i>Рис. из <a href=$Математические методы обучения по прецедентам$>лекций К. Воронцова</a> по машинному обучению</i></center>

### Kernel PCA

PCA (principal component analysis, метод главных компонент) - один из способ сокращения размерности данных, при котором в пространстве признаков осуществляется поиск нового упорядоченного базиса. Подробнее про этот метод можно прочитать, например, [здесь](https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%B3%D0%BB%D0%B0%D0%B2%D0%BD%D1%8B%D1%85_%D0%BA%D0%BE%D0%BC%D0%BF%D0%BE%D0%BD%D0%B5%D0%BD%D1%82).

Идея метода kernel PCA (principal component analysis) заключается в том, что сначала вектора признаков отображаются в спрямляющее пространство, как было описано выше, а затем выполняется линейный PCA в этом пространстве. Как и в SVM, в kernel PCA значения $\Phi(x)$ не рассчитываются в явном виде, потому что спрямляющее пространство может иметь очень высокую размерность или даже быть бесконечномерным. Вместо этого для расчета главных компонент используются ядра $k(x, x')$. Про метод kernel PCA можно прочитать в [оригинальной статье]($Nonlinear Component Analysis as a Kernel Eigenvalue Problem$) (1996), а также [здесь]($Kernel Principal Component Analysis$) и [здесь]($Kernel Principal Component Analysis and its Applications in Face Recognition and Active Shape Models$).

<img src="assets/kernel_pca.jpg" width="500" align="center">

<center><i>Рис. из статьи Kernel Principal Component Analysis (1997)</i></center>

Более подробное изложение ядерных методов можно найти в следующих источниках:
- [Лекции Константина Воронцова по машинному обучению]($Математические методы обучения по прецедентам$), §4.5 "Метод опорных вектов"
- Книга [Learning with Kernels]($Learning with Kernels$) (B Schölkopf, A Smola, 2002); глава 1 "A Tutorial Introduction" и далее
- Статья [Support-Vector Networks]($Support-Vector Networks$) (C Cortes, V Vapnik, 1995)
- Статья [A Tutorial on Support Vector Machines for Pattern Recognition]($A Tutorial on Support Vector Machines for Pattern Recognition$) (C Burges, 1998)
- Статья [A Tutorial on Support Vector Regression]($A Tutorial on Support Vector Regression$)  (A Smola, B Schölkopf, 2004)
- Статья [An Idiot’s guide to Support vector machines (SVMs)](http://web.mit.edu/6.034/wwwbob/svm-notes-long-08.pdf) (R Berwick, 2003)